In [1]:
!pip install -q sentence-transformers faiss-cpu transformers gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 64.1 MB/s eta 0:00:00


In [2]:
import json

law_files = [
    "salary_laws.json",
    "termination_laws.json",
    "harassment_laws.json",
    "benefits_laws.json",
    "overtime_laws.json"
]

with open("action.json","r") as f:
    actions_data = json.load(f)

laws_data = []

for file in law_files:
    with open(file,"r") as f:
        data = json.load(f)
        laws_data.extend(data)

print("Total law records:", len(laws_data))
print("Action categories:", actions_data.keys())

Total law records: 30
Action categories: dict_keys(['Salary Delay', 'Unfair Termination', 'Workplace Harassment', 'Denial of Benefits', 'Overtime Violation'])


In [3]:
import json

file_path = "salary_laws.json"

try:
    with open(file_path, "r", encoding="utf-8") as f:
        content = f.read()

    # The error message indicated 'char 588' as the problematic position.
    # We will attempt to insert a comma at this exact character index.
    # This is a highly specific fix based on the error message. If the JSON
    # is malformed in a more complex way, this might not be sufficient.

    fixed_content = content[:588] + ',' + content[588:]

    # Try to load the fixed content to validate it
    fixed_data = json.loads(fixed_content)

    # If successful, overwrite the original file with the corrected JSON
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(fixed_data, f, indent=2) # Use indent for readability

    print(f"Successfully fixed '{file_path}' by inserting a comma at char 588.")
    print("Please now re-run the cell where salary_laws.json was being loaded (cell hYhCGCaKwp4c).")

except json.JSONDecodeError as e:
    print(f"Attempted to fix '{file_path}', but a new JSONDecodeError occurred: {e}")
    print("The automatic fix might not have been sufficient. Please inspect the file manually around the indicated line.")
    print("\n--- Content around the error location ---")
    lines = fixed_content.splitlines()
    # Try to find the line from the new error, or default to the original problematic line
    error_lineno = e.lineno if hasattr(e, 'lineno') else 16 # Fallback if no new lineno
    start_line = max(0, error_lineno - 5)
    end_line = min(len(lines), error_lineno + 5)
    for i in range(start_line, end_line):
        line_indicator = ">>>" if i == error_lineno - 1 else "   "
        print(f"{line_indicator} {i+1}: {lines[i]}")
    print("------------------------------------------")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


Attempted to fix 'salary_laws.json', but a new JSONDecodeError occurred: Expecting property name enclosed in double quotes: line 16 column 7 (char 589)
The automatic fix might not have been sufficient. Please inspect the file manually around the indicated line.

--- Content around the error location ---
    12:       "salary",
    13:       "wages",
    14:       "payment",
    15:       "delay"
>>> 16:     ],,
    17:     "confidence_notes": "Official IndiaCode PDF."
    18:   },
    19:   {
    20:     "id": "salary_02",
    21:     "category": "Salary Delay",
------------------------------------------


In [4]:
import json

file_path = "action.json"
try:
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    print("action.json is valid JSON.")
except json.JSONDecodeError as e:
    print(f"JSONDecodeError: {e}")
    print(f"Error occurred at character {e.pos} on line {e.lineno}.")
    with open(file_path, "r", encoding="utf-8") as f:
        content = f.read()
        lines = content.splitlines()
        print("\n--- Content around the error location ---")
        start_line = max(0, e.lineno - 5)
        end_line = min(len(lines), e.lineno + 5)
        for i in range(start_line, end_line):
            line_indicator = ">>>" if i == e.lineno - 1 else "   "
            print(f"{line_indicator} {i+1}: {lines[i]}")
        print("------------------------------------------")
    print("\nPlease examine the content printed above. The error 'Extra data' usually means there is content after a complete JSON object/array.")
    print("You might need to manually edit 'action.json' to ensure it contains only one valid JSON structure. If it was intended to be a single dictionary, ensure there are no trailing characters after the final '}'.")


action.json is valid JSON.


In [5]:
law_texts = []
law_metadata = []

for item in laws_data:
    text = f"{item['category']} {item['law_name']} {item['section']} {item['summary']}"
    law_texts.append(text)
    law_metadata.append(item)

In [6]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

embedder = SentenceTransformer("all-MiniLM-L6-v2")

embeddings = embedder.encode(law_texts)

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(np.array(embeddings))

print("Vector DB built.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Vector DB built.


In [7]:
from transformers import pipeline

generator = pipeline(
    "text-generation",
    model="google/flan-t5-base",
    max_new_tokens=200
)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/282 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Passing `generation_config` together with generation-related arguments=({'max_new_tokens'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['PeftModelForCausalLM', 'AfmoeForCausalLM', 'ApertusForCausalLM', 'ArceeForCausalLM', 'AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BitNetForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'BltForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'CwmForCausalLM', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV2ForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaF

In [8]:
def detect_category(text):
    text = text.lower()
    if any(w in text for w in ["salary","wage","pay"]):
        return "Salary Delay"
    elif any(w in text for w in ["fired","terminated","dismissed"]):
        return "Unfair Termination"
    elif any(w in text for w in ["harass","sexual","abuse"]):
        return "Workplace Harassment"
    elif any(w in text for w in ["pf","esi","gratuity","benefit"]):
        return "Denial of Benefits"
    elif any(w in text for w in ["overtime","extra hours"]):
        return "Overtime Violation"
    else:
        return None

In [9]:
def retrieve_law(user_query):
    q_emb = embedder.encode([user_query])
    D,I = index.search(np.array(q_emb),1)
    return law_metadata[I[0][0]]

def get_action_block(category):
    return actions_data.get(category)

In [10]:
def generate_email(issue, steps):
    prompt = f"""
You are writing a professional legal complaint email.

Write ONLY the email content.
Do NOT repeat instructions.
Do NOT explain anything.

Format:

Subject: ...
Body:
...

Issue: {issue}

Steps to mention: {steps}
"""

    result = generator(prompt, max_new_tokens=150)[0]["generated_text"]

    # ✂️ HARD CLEANUP (removes instruction leakage)
    lines = result.split("\n")
    clean_lines = []
    for line in lines:
        if not any(word in line.lower() for word in ["issue:", "steps", "only output", "write"]):
            clean_lines.append(line)

    return "\n".join(clean_lines).strip()

In [11]:
def generate_legal_response(user_problem):
    category = detect_category(user_problem)
    if not category:
        return "❌ Unable to detect category. Please describe your problem clearly."

    law = retrieve_law(user_problem)
    action_block = get_action_block(category)

    steps = "\n".join(action_block["steps"])
    documents = "\n".join(action_block["documents_required"])
    portal = action_block["official_portals"][0]["url"]

    email = generate_email(law["summary"], steps)

    output = f"""
🗂 CATEGORY: {category}

📌 ISSUE:
{action_block["description"]}

⚖️ RELEVANT LAW:
- {law["law_name"]} ({law["section"]})

🪜 NEXT STEPS:
{steps}

📄 DOCUMENTS REQUIRED:
{documents}

✉️ DRAFT EMAIL:
{email}

🌐 FILE COMPLAINT HERE:
{portal}

🤝 CONCLUSION:
We understand this situation can be stressful and upsetting. You are not alone, and the law provides protection for employees in such cases. Following the above steps will help you take proper legal action calmly and correctly.

⚠️ DISCLAIMER:
This is only preliminary legal guidance. Please consult a qualified lawyer.
"""
    return output

In [12]:
print(generate_legal_response("I was fired without notice"))

Both `max_new_tokens` (=150) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



🗂 CATEGORY: Unfair Termination

📌 ISSUE:
Employee dismissed without notice, compensation, or valid procedure.

⚖️ RELEVANT LAW:
- Industrial Disputes Act, 1947 (Retrenchment and notice)

🪜 NEXT STEPS:
Step 1: Request written termination reason from employer.
Step 2: Review employment contract and termination clause.
Step 3: File complaint under Industrial Disputes Act.
Step 4: Approach Labour Court or Industrial Tribunal.

📄 DOCUMENTS REQUIRED:
Offer letter
Termination letter
Experience certificate
Salary records

✉️ DRAFT EMAIL:
You are writing a professional legal complaint email.

Do NOT repeat instructions.
Do NOT explain anything.

Format:

Subject: ...
Body:
...


Step 2: Review employment contract and termination clause.
Step 3: File complaint under Industrial Disputes Act.
Step 4: Approach Labour Court or Industrial Tribunal.

🌐 FILE COMPLAINT HERE:
https://labour.gov.in

🤝 CONCLUSION:
We understand this situation can be stressful and upsetting. You are not alone, and the law 

In [13]:
# =========================================================
# SMART EMAIL GENERATOR (category specific + varied)
# =========================================================
def generate_email(category, issue=""):

    templates = {

        "Salary Delay": f"""Subject: Request for Release of Pending Salary

Dear Sir/Madam,

This is to inform you that my salary has not been paid within the prescribed time period.

{issue}

The delay has caused financial hardship and disruption to my regular expenses. I request immediate payment of all outstanding dues.

Sincerely,
[Your Name]
""",

        "Denial of Benefits": f"""Subject: Non-deposit of Statutory Benefits (PF/ESI/Gratuity)

Dear Sir/Madam,

My statutory employment benefits, including provident fund and related contributions, have not been deposited as required by law.

{issue}

Kindly review the records and ensure immediate correction.

Sincerely,
[Your Name]
""",

        "Unfair Termination": f"""Subject: Representation Against Unfair Termination

Dear Sir/Madam,

I have been terminated from my employment without proper notice or due procedure.

{issue}

I respectfully request a written explanation and review of this action.

Sincerely,
[Your Name]
""",

        "Workplace Harassment": f"""Subject: Formal Complaint Regarding Workplace Harassment

Dear Sir/Madam,

I would like to formally report harassment at the workplace.

{issue}

I request that this matter be investigated promptly and appropriate action be taken.

Sincerely,
[Your Name]
""",

        "Overtime Violation": f"""Subject: Claim for Unpaid Overtime Compensation

Dear Sir/Madam,

I have been required to work beyond regular working hours without receiving overtime wages.

{issue}

The continuous extra workload has resulted in fatigue and mental distress. I request settlement of my overtime dues.

Sincerely,
[Your Name]
"""
    }

    return templates.get(category, "Please describe your issue clearly.")

In [14]:
category = "Unfair Termination"  # Example value
issue = "I was fired without notice"  # Example value, can be a summary from law['summary']
email_text = generate_email(category, issue)

In [15]:
print(generate_legal_response("I was fired without notice"))


🗂 CATEGORY: Unfair Termination

📌 ISSUE:
Employee dismissed without notice, compensation, or valid procedure.

⚖️ RELEVANT LAW:
- Industrial Disputes Act, 1947 (Retrenchment and notice)

🪜 NEXT STEPS:
Step 1: Request written termination reason from employer.
Step 2: Review employment contract and termination clause.
Step 3: File complaint under Industrial Disputes Act.
Step 4: Approach Labour Court or Industrial Tribunal.

📄 DOCUMENTS REQUIRED:
Offer letter
Termination letter
Experience certificate
Salary records

✉️ DRAFT EMAIL:
Please describe your issue clearly.

🌐 FILE COMPLAINT HERE:
https://labour.gov.in

🤝 CONCLUSION:
We understand this situation can be stressful and upsetting. You are not alone, and the law provides protection for employees in such cases. Following the above steps will help you take proper legal action calmly and correctly.

⚠️ DISCLAIMER:
This is only preliminary legal guidance. Please consult a qualified lawyer.



In [16]:
pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 56.3 MB/s eta 0:00:00
  Attempting uninstall: cachetools
    Found existing installation: cachetools 7.0.1
    Uninstalling cachetools-7.0.1:
      Successfully uninstalled cachetools-7.0.1


In [17]:
import streamlit as st
import json
import random


# ======================================================
# PAGE CONFIG
# ======================================================
st.set_page_config(
    page_title="Employee Legal Assistant",
    page_icon="⚖️",
    layout="centered"
)


# ======================================================
# DARK THEME + SMALL ANIMATIONS
# ======================================================
st.markdown("""
<style>

.stApp {
    background: linear-gradient(135deg,#0f172a,#1e293b);
    color:#e2e8f0;
    font-size:17px;
}

h1 {
    font-size:48px !important;
    text-align:center;
    font-weight:800;
    color:#60a5fa;
}

.block {
    background:#1e293b;
    padding:18px;
    border-radius:14px;
    margin-bottom:14px;
    border:1px solid #334155;
    animation:fadeIn 0.4s ease;
}

@keyframes fadeIn {
    from {opacity:0; transform:translateY(8px);}
    to {opacity:1; transform:translateY(0);}
}

</style>
""", unsafe_allow_html=True)


# ======================================================
# LOAD JSON FILES
# ======================================================
def load_json(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)


actions_data = load_json("action.json")
salary_laws = load_json("salary_laws.json")
termination_laws = load_json("termination_laws.json")
harassment_laws = load_json("harassment_laws.json")
benefits_laws = load_json("benefits_laws.json")
overtime_laws = load_json("overtime_laws.json")


laws_map = {
    "Salary Delay": salary_laws,
    "Unfair Termination": termination_laws,
    "Workplace Harassment": harassment_laws,
    "Denial of Benefits": benefits_laws,
    "Overtime Violation": overtime_laws
}


# ======================================================
# CLASSIFIER + CONFIDENCE (75–90 ONLY)
# ======================================================
def classify_issue(text):

    text = text.lower()

    categories = {

        "Workplace Harassment": [
            "harass","harassment","bully","abuse","threat",
            "sexual","misconduct","hostile","toxic"
        ],

        "Salary Delay": [
            "salary","wage","wages","pay","not paid",
            "unpaid","salary delay","late salary","pending salary"
        ],

        "Overtime Violation": [
            "overtime","extra hours","late night","worked late",
            "extended hours","night shift","no overtime pay","unpaid overtime"
        ],

        "Denial of Benefits": [
            "pf","epf","esi","gratuity","benefit","insurance","not credited"
        ],

        "Unfair Termination": [
            "fired","terminated","dismissed","removed",
            "laid off","layoff","downsized","job cut","sacked"
        ]
    }

    best_cat = None
    best_score = 0

    for cat, words in categories.items():
        score = sum(1 for w in words if w in text)
        if score > best_score:
            best_cat = cat
            best_score = score

    if not best_cat:
        return None, 0

    # confidence range fixed between 75–90
    confidence = 75 + min(best_score, 3) * 5
    confidence = min(confidence, 90)

    return best_cat, confidence


# ======================================================
# ANIMATED CONFIDENCE BAR
# ======================================================
def show_confidence(score):

    bar = st.progress(0)

    for i in range(score):
        bar.progress(i + 1)

    st.markdown(f"**Model Confidence: {score}%**")


# ======================================================
# EMAIL
# ======================================================
def generate_email(category, issue):

    return f"""Subject: Formal Complaint – {category}

Dear Sir/Madam,

I am writing to formally report that {issue.lower()}.

I kindly request that this matter be reviewed and appropriate action be taken in accordance with applicable labour laws.

Sincerely,
[Your Name]
"""


# ======================================================
# PROCESS QUERY
# ======================================================
def process_query(user_problem):

    category, confidence = classify_issue(user_problem)

    if not category:
        return None

    law_data = laws_map.get(category, [])
    action = actions_data.get(category, {})

    issue = action.get("description", "")

    laws = "\n".join(
        [f"- {l.get('law_name')} ({l.get('section','')})" for l in law_data]
    )

    steps = "\n\n".join(
        [f"Step {i+1}: {s}" for i, s in enumerate(action.get("steps", []))]
    )

    docs = "\n".join(action.get("documents_required", []))

    portals = action.get("official_portals", [])
    url = portals[0]["url"] if portals else "#"

    email = generate_email(category, issue)

    conclusion = random.choice([
        "Please stay confident. The law protects employees and there are clear remedies available.",
        "You are not alone in this situation. Following the steps above can help resolve the issue calmly.",
        "With the right documentation and action, you can move forward with confidence.",
        "Take it one step at a time — support and legal protections are available to you."
    ])

    return category, confidence, issue, laws, steps, docs, email, url, conclusion


# ======================================================
# UI
# ======================================================
st.title("⚖️ Employee Legal Assistant")
st.write("Describe your workplace issue below:")

user_input = st.text_area("Enter your legal issue")


# ======================================================
# BUTTON
# ======================================================
if st.button("Get Help"):

    result = process_query(user_input)

    if not result:
        st.error("Please describe a valid workplace issue.")
        st.stop()

    category, confidence, issue, laws, steps, docs, email, url, conclusion = result

    show_confidence(confidence)

    st.markdown(f"<div class='block'><b>📌 Issue</b><br>{issue}</div>", unsafe_allow_html=True)
    st.markdown(f"<div class='block'><b>⚖️ Relevant Laws</b><br>{laws}</div>", unsafe_allow_html=True)
    st.markdown(f"<div class='block'><b>🪜 Steps</b><br>{steps}</div>", unsafe_allow_html=True)
    st.markdown(f"<div class='block'><b>📄 Documents</b><br>{docs}</div>", unsafe_allow_html=True)

    edited = st.text_area("✉️ Draft Email (Editable)", email, height=220)
    st.download_button("Download Email", edited)

    st.markdown(f"<div class='block'><b>🌐 Official Portal</b><br><a href='{url}' target='_blank'>{url}</a></div>", unsafe_allow_html=True)

    st.markdown(f"<div class='block'><b>🤝 Conclusion</b><br>{conclusion}</div>", unsafe_allow_html=True)

    st.caption("⚠️ This is preliminary legal guidance only. Please consult a qualified lawyer.")

2026-02-22 04:20:41.193 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-22 04:20:41.194 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-22 04:20:41.363 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2026-02-22 04:20:41.365 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-22 04:20:41.365 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-22 04:20:41.370 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-22 04:20:41.372 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn

In [22]:
%%writefile app.py
# (paste your streamlit code here)
import streamlit as st
import json
import random


# =====================================================
# LOAD JSON
# =====================================================
def load_json(file):
    with open(file, "r", encoding="utf-8") as f:
        return json.load(f)


actions_data = load_json("action.json")
salary_laws = load_json("salary_laws.json")
termination_laws = load_json("termination_laws.json")
harassment_laws = load_json("harassment_laws.json")
benefits_laws = load_json("benefits_laws.json")
overtime_laws = load_json("overtime_laws.json")


laws_map = {
    "Salary Delay": salary_laws,
    "Unfair Termination": termination_laws,
    "Workplace Harassment": harassment_laws,
    "Denial of Benefits": benefits_laws,
    "Overtime Violation": overtime_laws,
}


# =====================================================
# SMART UNIVERSAL CLASSIFIER (phrase based + realistic)
# =====================================================
def classify_issue(text):
    text = text.lower()

    categories = {
        "Workplace Harassment": [
            "harass", "harassment", "abuse", "bully", "threat", "sexual"
        ],

        "Salary Delay": [
            "salary delay", "not paid", "salary not paid",
            "wage delay", "payment delay", "unpaid salary"
        ],

        "Overtime Violation": [
            "overtime",
            "extra hours",
            "late night",
            "late nights",
            "worked late",
            "working late",
            "after office",
            "beyond hours",
            "extended hours",
            "extra shift",
            "no compensation",
            "no overtime pay",
            "night shift unpaid"
        ],

        "Denial of Benefits": [
            "pf", "epf", "benefit", "gratuity", "insurance", "esi"
        ],

        "Unfair Termination": [
            "fired", "terminate", "terminated", "dismiss",
            "removed", "laid off"
        ]
    }

    for category, phrases in categories.items():
        for phrase in phrases:
            if phrase in text:
                return category

    return None


# =====================================================
# EMAIL GENERATOR (custom per case)
# =====================================================
def generate_email(category):

    emails = {

        "Salary Delay": [
            """Subject: Request for Pending Salary Payment

Dear Sir/Madam,

My salary has not been credited within the scheduled time, causing financial inconvenience. I request immediate release of the outstanding amount.

Sincerely,
[Your Name]
"""
        ],

        "Denial of Benefits": [
            """Subject: Non-deposit of Statutory Benefits

Dear Sir/Madam,

My statutory benefits such as provident fund/insurance contributions have not been deposited as required. Kindly review and ensure compliance at the earliest.

Sincerely,
[Your Name]
"""
        ],

        "Unfair Termination": [
            """Subject: Representation Against Termination

Dear Sir/Madam,

My employment has been terminated without proper notice or due procedure. I request clarification and reconsideration of this decision.

Sincerely,
[Your Name]
"""
        ],

        "Workplace Harassment": [
            """Subject: Complaint Regarding Workplace Harassment

Dear Sir/Madam,

I wish to report conduct at the workplace that has created an uncomfortable and unsafe environment. Kindly investigate and take appropriate action.

Sincerely,
[Your Name]
"""
        ],

        "Overtime Violation": [
            """Subject: Claim for Unpaid Overtime

Dear Sir/Madam,

I have been required to work beyond regular hours without receiving overtime compensation. The prolonged workload has caused fatigue and mental distress. I request settlement of my overtime dues.

Sincerely,
[Your Name]
"""
        ]
    }

    return random.choice(emails.get(category))


# =====================================================
# EMPATHETIC CONCLUSION
# =====================================================
def generate_conclusion(category):

    lines = [
        f"This appears to be a {category} issue. Please remember that you are legally protected and support is available.",
        f"We understand that situations like {category} can feel stressful. Taking the right steps can help you resolve this confidently.",
        f"You deserve fair treatment at work. Addressing this {category} concern through proper channels can help achieve a positive outcome.",
        f"Workplace concerns can be difficult, but there are legal remedies to assist you in resolving this matter."
    ]

    return random.choice(lines)


# =====================================================
# CORE LOGIC
# =====================================================
def process_query(user_input):

    category = classify_issue(user_input)

    if not category:
        return None

    law_data = laws_map.get(category, [])
    action_data = actions_data.get(category, {})

    issue = action_data.get("description", f"{category} related legal issue.")

    laws = []
    for item in law_data:
        name = item.get("law_name") or item.get("law")
        if name:
            section = item.get("section", "")
            laws.append(f"- {name} ({section})")

    laws = "\n".join(laws)

    steps = "\n\n".join(
        [f"Step {i+1}: {s}" for i, s in enumerate(action_data.get("steps", []))]
    )

    documents = action_data.get("documents_required", [])
    docs = "\n".join([f"- {d}" for d in documents])

    portals = action_data.get("official_portals", [])
    url = portals[0]["url"] if portals else "https://labour.gov.in"

    email = generate_email(category)
    conclusion = generate_conclusion(category)

    return category, issue, laws, steps, docs, email, url, conclusion


# =====================================================
# STREAMLIT UI
# =====================================================
st.set_page_config(page_title="Employee Legal Assistant", page_icon="⚖️")

st.title("⚖️ Employee Legal Assistant")
st.write("Describe your workplace problem in simple words:")

user_input = st.text_area("Enter your legal issue:")


if st.button("Get Help"):

    if not user_input.strip():
        st.warning("Please enter your issue.")
        st.stop()

    result = process_query(user_input)

    if not result:
        st.error("❌ Please describe a valid workplace issue (salary, PF, harassment, termination, overtime).")
        st.stop()

    category, issue, laws, steps, docs, email, url, conclusion = result

    st.markdown(f"## 🗂 CATEGORY: {category}")

    st.markdown("### 📌 ISSUE:")
    st.write(issue)

    st.markdown("### ⚖️ RELEVANT LAW:")
    st.write(laws)

    st.markdown("### 🪜 NEXT STEPS:")
    st.write(steps)

    st.markdown("### 📄 DOCUMENTS REQUIRED:")
    st.write(docs)

    st.markdown("### ✉️ DRAFT EMAIL:")
    st.text(email)

    st.markdown("### 🌐 FILE COMPLAINT HERE:")
    st.markdown(url)

    st.markdown("### 🤝 CONCLUSION:")
    st.write(conclusion)

    st.markdown("### ⚠️ DISCLAIMER:")
    st.write("This is preliminary legal guidance. Please consult a qualified lawyer.")

Writing app.py


In [18]:
!pip install streamlit pyngrok

In [19]:
pip install streamlit-lottie

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 12.9 MB/s eta 0:00:00


In [33]:
!streamlit run app.py &>/content/logs.txt &

In [34]:
from pyngrok import ngrok

ngrok.set_auth_token("39z924FKF5eOsfkR9z59Y14T3qF_24c1Nvngw5docdWJRzuRJ")

# close any existing tunnels first
for t in ngrok.get_tunnels():
    ngrok.disconnect(t.public_url)

public_url = ngrok.connect(8501)

print("Streamlit running at:", public_url)

Streamlit running at: NgrokTunnel: "https://smeariest-unsurfaced-mallie.ngrok-free.dev" -> "http://localhost:8501"
